In [1]:
# Note: Here are some things to consider in your notebook: sample size, correlations, feature importance, unexplained variance or outliers, variable selection, imbalanced dataset (if classification), and any relationships between your target and independent variables.

In [1]:
# INPUT
year = 2021
# Select from 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms'
# ['won_award', 'danceability', 'energy', 'key', 'loudness', 'mode','speechiness', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo', 'duration_ms', 'time_signature', 'Category', 'Cat','musician', 'song']
trait = 'danceability'

In [2]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
df = pd.read_csv('grammy_nominees_with_music_attributes.csv').drop('Unnamed: 0', axis=1)
df_history = df[(df['year'] < year) & (df['year'] > year - 10)]
df_this_year = df[df['year'] == year]
df_this_year['Category'] = 'Current Year (Loser)'
df_this_year['Cat'] = 'Historic'
df_this_year['song'] = df_this_year['song']
df_this_year.loc[df_this_year['won_award'] == 1, 'Category'] = 'Current Year (Winner)'

In [9]:
past_winners = df_history.drop(columns=['year']).groupby('won_award').agg('mean').reset_index()
# past_winners_25th_percentile = df_history.drop(columns=['year']).groupby('won_award').quantile(q=.25).reset_index()
# past_winners_75th_percentile = df_history.drop(columns=['year']).groupby('won_award').quantile(q=.75).reset_index()
past_winners['Category'] = 'Historic'
past_winners['Cat'] = 'Historic'
past_winners['musician'] = ''
past_winners.loc[past_winners['won_award'] == 0, 'Category'] = 'Historic Losers (10 years)'
past_winners.loc[past_winners['won_award'] == 1, 'Category'] = 'Historic Winners (10 years)'
past_winners['song'] = past_winners['Category']

In [10]:
compare_nominees_to_history = pd.concat([past_winners,df_this_year[['song', 'musician', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature', 'Category', 'won_award']]]).set_index('song', drop=True)

In [32]:
grouped = df.groupby('year').mean()
grouped = grouped[['speechiness', 'acousticness', 'duration_ms']]
grouped[['speechiness', 'acousticness', 'duration_ms']] = MinMaxScaler().fit_transform(grouped[['speechiness', 'acousticness', 'duration_ms']])
grouped = grouped.rolling(10).mean().dropna()

In [13]:
# music over the radio, not as clear, 
# lyrics now are the most importnat part because people conenct with the musician
# song-writers are more importnat now than they have ever been
# rise of rap and hip hop
# billboard top 100 charts
# dot com bubble

In [14]:
characteristics = df_history.corr()['won_award'].sort_values(ascending=False).index
important_characteristics = list(characteristics[1:4]) + list(characteristics[-3:])
important_characteristics

['danceability', 'mode', 'speechiness', 'energy', 'key', 'tempo']

In [20]:
list(compare_nominees_to_history[important_characteristics].iloc[0])

[0.6170526315789475,
 0.7368421052631579,
 0.08071315789473683,
 0.5766578947368421,
 6.0,
 122.1117894736842]

In [34]:
# df_comparison = compare_nominees_to_history.set_index('song')
compare_nominees_to_history.loc["I Can't Breathe"][important_characteristics]

danceability      0.703
mode                0.0
speechiness        0.34
energy            0.705
key                 1.0
tempo           138.227
Name: I Can't Breathe, dtype: object

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
mask = np.zeros_like(df_history.corr())
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(15, 10))
sns.heatmap(
    df_history.corr(),
    cmap='coolwarm',
    annot=True,
    mask=mask,
    vmin=-1.,
    vmax=1.
);

NameError: name 'df_history' is not defined

In [11]:
X_train = df_history[important_characteristics]
y_train = df_history['won_award']
X_test = df_this_year[important_characteristics]
y_test = df_this_year['won_award']

In [12]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train, y_train)
X_test_scaled = ss.transform(X_test)

In [13]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures()
X_train_expanded = pf.fit_transform(X_train, y_train)
X_test_expanded = pf.transform(X_test)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
params = {'C': [.001, .01, .1, 1, 10, 100, 1000]}
gs_raw = GridSearchCV(LogisticRegression(), params, n_jobs=-1)
gs_raw.fit(X_train, y_train)
raw_preds = list(gs_raw.predict_proba(X_test))
# predictions = pd.DataFrame({'song':df_this_year['song'], 'predictions': raw_preds})

In [15]:
params = {'C': [.001, .01, .1, 1, 10, 100, 1000]}
gs_scaled = GridSearchCV(LogisticRegression(), params, n_jobs=-1)
gs_scaled.fit(X_train_scaled, y_train)
gs_scaled.score(X_test_scaled, y_test)
scaled_preds = list(gs_scaled.predict_proba(X_test_scaled))
# predictions = pd.DataFrame({'song':df_this_year['song'], 'predictions': scaled_preds})

In [16]:
params = {'C': [.001, .01, .1, 1, 10, 100, 1000]}
gs_pf = GridSearchCV(LogisticRegression(), params, n_jobs=-1)
gs_pf.fit(X_train_expanded, y_train)
gs_pf.predict(X_test_expanded)
pf_preds = list(gs_pf.predict_proba(X_test_expanded))
predictions = pd.DataFrame({'song':df_this_year['song'], 'predictions': pf_preds})

/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [17]:
predictions['chris_prob_win'] = predictions['predictions'].apply(lambda x: x[1])
predictions = predictions[['song', 'chris_prob_win']]
total_prob_win = predictions['chris_prob_win'].sum()
predictions['chris_prob_win'] = predictions['chris_prob_win']/total_prob_win
predictions.sort_values(by='chris_prob_win', ascending=False, inplace=True)
predictions.reset_index(drop=True, inplace=True)

# Get betting odds from https://www.covers.com/entertainment/grammy-awards-betting-odds-picks-best-bets
betting_odds = pd.DataFrame(columns=['song', 'odds_as_of_jan_22_2021'])
betting_odds.loc[len(betting_odds)] = ['Black Parade', '+300']
betting_odds.loc[len(betting_odds)] = ["Don't Start Now", '+333']
betting_odds.loc[len(betting_odds)] = ['Cardigan', '+350']
betting_odds.loc[len(betting_odds)] = ['Everything I Wanted', '+1,000']
betting_odds.loc[len(betting_odds)] = ['If the World Was Ending', '+1,600']
betting_odds.loc[len(betting_odds)] = ['The Box', '+1,800']
betting_odds.loc[len(betting_odds)] = ["I Can't Breathe", '+2,500']
betting_odds.loc[len(betting_odds)] = ['Circles', '+1,200']
def convert_betting_line_to_implied_odds(line):
    favorite = True if line[0] == '+' else False
    line = int(line[1:].replace(',', ''))
    if favorite:
        return 100 / (line + 100)
    else:
        return line / (line + 100)
betting_odds['covers_com_implied_odds'] = betting_odds['odds_as_of_jan_22_2021'].apply(convert_betting_line_to_implied_odds)
betting_odds.drop(columns=['odds_as_of_jan_22_2021'], inplace=True)
predictions = pd.merge(predictions, betting_odds, on='song')
predictions.set_index('song', inplace=True)
predictions['chris_prob_win'] = predictions['chris_prob_win'].apply(lambda x: str(round(x * 100, 2)) + '%')
predictions['covers_com_implied_odds'] = predictions['covers_com_implied_odds'].apply(lambda x: str(round(x * 100, 2)) + '%')
predictions

,chris_prob_win,implied_odds
song,,
If the World Was Ending,30.25%,5.88%
Black Parade,29.82%,25.0%
I Can't Breathe,7.0%,3.85%
Everything I Wanted,5.82%,9.09%
Cardigan,5.01%,22.22%
The Box,1.77%,5.26%
Don't Start Now,0.42%,23.09%


In [2]:
import streamlit
streamlit.__version__

'0.72.0'